<a href="https://colab.research.google.com/github/samp3209/wineproject/blob/main/winemodeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('wine - final (1).csv')

In [13]:
df

,wine_names,year,price,region,country,quantity,pp750ml
0,"Vintage Port, Douro, Portugal, Sandeman",2020.0,323.36,duoro,port,750 ml,323.3600
1,"Vintage Port, Douro, Portugal, Sandeman",2020.0,93.17,duoro,port,750 ml,93.1700
2,"Quinta do Vale Meao ""tinto"", Douro magum op ki...",2019.0,284.45,duoro,port,750 ml,284.4500
3,Ferreira Vintage Port Douro 2019,2019.0,92.78,duoro,port,750 ml,92.7800
4,"Quinta do Vale Meao Tinto, Portugal, Douro",2019.0,115.10,duoro,port,750 ml,115.1000
...,...,...,...,...,...,...,...
9264,"Caymus Cabernet Sauvignon 2020 1L, USA, CALIFO...",2020.0,89.95,cali,cali,1x1000 ml,67.4625
9265,2020 Caymus Cabernet Sauvignon Napa Valley 1L,2020.0,89.94,cali,cali,1x1000 ml,67.4550
9266,"Caymus Cabernet Sauvignon Napa Valley, 1L",2020.0,89.89,cali,cali,1x1000 ml,67.4175
9267,"Caymus Vineyards Cabernet Sauvignon 2020, 1L",2020.0,84.95,cali,cali,1x1000 ml,63.7125


In [14]:
# one hot encode the region and country columns
region_dummies = pd.get_dummies(df['region'], prefix='region')
country_dummies = pd.get_dummies(df['country'], prefix='country')

# concatenate the original dataframe with the one hot encoded columns
df = pd.concat([df, region_dummies, country_dummies], axis=1)

# drop the original region and country columns
df = df.drop(['region', 'country'], axis=1)

# show the resulting dataframe
print(df)

                                             wine_names    year   price  \
0               Vintage Port, Douro, Portugal, Sandeman  2020.0  323.36   
1               Vintage Port, Douro, Portugal, Sandeman  2020.0   93.17   
2     Quinta do Vale Meao "tinto", Douro magum op ki...  2019.0  284.45   
3                      Ferreira Vintage Port Douro 2019  2019.0   92.78   
4            Quinta do Vale Meao Tinto, Portugal, Douro  2019.0  115.10   
...                                                 ...     ...     ...   
9264  Caymus Cabernet Sauvignon 2020 1L, USA, CALIFO...  2020.0   89.95   
9265      2020 Caymus Cabernet Sauvignon Napa Valley 1L  2020.0   89.94   
9266          Caymus Cabernet Sauvignon Napa Valley, 1L  2020.0   89.89   
9267       Caymus Vineyards Cabernet Sauvignon 2020, 1L  2020.0   84.95   
9268             Caymus Napa Cabernet 1.0L, Napa Valley  2020.0   81.99   

       quantity   pp750ml  region_Rheinhessen  region_algarve  region_alsace  \
0        750 ml  32

In [14]:
from sklearn.model_selection import train_test_split
import numpy as np 


In [15]:
from sklearn.linear_model import LinearRegression

# Initialize the model
model = LinearRegression()

# Fit the model to your data
model.fit(X_train, y_train)

# Make predictions on new data
y_pred = model.predict(X_test)

NameError: ignored

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Initialize the model
model = DecisionTreeRegressor()

# Fit the model to your data
model.fit(X_train, y_train)

# Make predictions on new data
y_pred = model.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
model = RandomForestRegressor()

# Fit the model to your data
model.fit(X_train, y_train)

# Make predictions on new data
y_pred = model.predict(X_test)

In [ ]:
from sklearn.svm import SVR

# Initialize the model
model = SVR()

# Fit the model to your data
model.fit(X_train, y_train)

# Make predictions on new data
y_pred = model.predict(X_test)